# 메모리관리: 대화 기록 유지
## 기본적인 대화 이력 전달
- 사용자가 이전에 어떤 질문을 했는지 기억하고 연관된 답변을 제공할 수 있습니다.
- 자연스럽고 맥락을 반영한 대화를 이어갈 수 있습니다.

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# AI 모델 정의
chat = ChatOpenAI(model="gpt-4o-mini")
# 프롬프트 템플릿 정의: 금융 상담 역할
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다."),
        ("placeholder", "{messages}"), #대화 이력 추가
    ]
)

- ChatPromptTemplate을 사용하여, 시스템 메시지를 포함한 프롬프트를 정의합니다.
- placeholder로 프롬프트에서 대화 이력이 추가될 위치를 설정합니다.

In [2]:
# 프롬프트와 모델을 연결하여 체인 생성
chain = prompt | chat

In [3]:
# 이전 대화를 포함한 메시지 전달
ai_msg = chain.invoke(
    {
        "messages": [
            ("human", "저축을 늘리기 위해 무엇을 할 수 있나요?"), # 사용자의 첫 질문
            ("ai", "저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요."), # 챗봇의 답변
            ("human", "방금 뭐라고 했나요?"), # 사용자의 재확인 질문
        ]
    }
)
print(ai_msg.content) # 챗봇의 답변 출력

저축을 늘리기 위해 목표를 설정하고, 매달 자동 이체를 통해 일정 금액을 저축하는 것을 권장했습니다. 이렇게 하면 저축이 더 쉬워지고, 소비 줄이기에 도움이 됩니다. 추가적으로, 필요 없는 지출을 줄이고, 예산을 세우는 것도 좋은 방법입니다.


## 대화 이력 관리 및 처리
- ChatMessageHistory 클래스를 사용하면 대화 내용을 저장하고 재사용하는 등, 대화 이력을 더 체계적으로 관리할수 있습니다.

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
# 대화 이력 저장을 위한 클래스 초기화
chat_history = ChatMessageHistory()
#사용자 메시지 추가
chat_history.add_user_message("저축을 늘리기 위해 무엇을 할 수 있나요?")
chat_history.add_ai_message("저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.")

- add_user_message와 add_ai_message 메서드를 사용해 대화 이력에 추가합니다.

In [5]:
# 새로운 질문 추가 후 다시 체인 실행
chat_history.add_user_message("방금 뭐라고 했나요?")
ai_response = chain.invoke({"messages": chat_history.messages})
print(ai_response.content) # 챗봇의 답변 출력

저축을 늘리기 위해 저축 목표를 설정하고 매달 자동 이체로 일정 금액을 저축할 것을 추천했습니다. 이렇게 하면 저축 습관을 형성하고 저축을 보다 쉽게 늘릴 수 있습니다. 추가적으로 다른 방법도 원하시면 말씀해 주세요!


## 자동 대화 이력 관리
- RunnableWithMessageHistory 클래스를 사용하면 대화 이력을 자동으로 관리할 수 있습니다.
- 이 클래스는 대화 이력을 자동으로 저장하고 불러오며, 이를 통해 이전 대화를 AI에게 전달하여 자연스러운 대화를 이어 나가게 합니다.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# 시스템 메시지와 대화 이력을 사용하는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오."),
        ("placeholder", "{chat_history}"), # 대화 이력 추가
        ("human", "{input}"), # 사용자의 질문 추가
    ]
)

- chat_history를 사용하여 이전 대화 이력을 포함시킵니다.

In [7]:
# 대화 이력을 관리할 체인 설정
chat_history = ChatMessageHistory()
chain = prompt | chat

- ChatMessageHistory 객체는 대화 이력을 저장하고 관리하는 역할을 합니다. 사용자의 이전 질문과 AI의 답변을 체계적으로 기록합니다.

In [8]:
# RunnalbleWithMessageHistory 클래스를 사용해 체인을 감쌉니다.
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history, # 세션 ID에 따라 대화 이력 관리
    input_messages_key="input", # 입력 메시지의 키 설정
    history_messages_key="chat_history", # 대화 이력의 키 설정
)

- RunnableWithMessageHistory 클래스를 사용하여 대화 이력을 자동으로 관리하는 체인을 설정합니다.
- 이 클래스를 사용하면 대화 이력을 저장하고 필요할 때 불러올 수 있습니다.
- session_id를 통해 특정 세션의 대화 이력을 추적하고, input_messages_key와 history_messages_key는 입력된 질문과 대화 이력을 처리하는 데 키를 정의합니다.

In [9]:
# 질문 메시지 체인 실행
chain_with_message_history.invoke(
    {"input": "저축을 늘리기 위해 무엇을 할 수 있나요?"},
    {"configurable": {"session_id": "unused"}}
).content

'저축을 늘리기 위해 여러 가지 방법을 고려할 수 있습니다. 아래는 몇 가지 유용한 팁입니다:\n\n1. **예산 세우기**: 자신의 수입과 지출을 분석하여 예산을 세우면 불필요한 지출을 줄이는 데 도움이 됩니다. 매달 얼마를 저축할 수 있는지 명확히 아는 것이 중요합니다.\n\n2. **자동 이체 설정**: 매달 일정 금액을 저축 계좌로 자동 이체하도록 설정하면, 저축을 잊지 않고 꾸준히 할 수 있습니다.\n\n3. **비필수 지출 줄이기**: 커피 한 잔, 외식, 유흥 비용 등을 줄이고, 절약한 금액을 저축에 사용하세요.\n\n4. **할인 및 쿠폰 활용**: 쇼핑할 때 쿠폰이나 할인 혜택을 활용하면, 지출을 줄이고 남은 돈을 저축할 수 있습니다.\n\n5. **지출 트래킹 앱 사용**: 지출을 기록하고 모니터링할 수 있는 앱을 사용하면 자신의 소비 패턴을 파악하고 필요 없는 부분을 조정할 수 있습니다.\n\n6. **비상금 저축**: 매달 일정 금액을 비상금으로 저축해두면 예상치 못한 지출에 대비할 수 있어 다른 자금으로부터 압박을 덜어줍니다.\n\n7. **목표 설정**: 장기적인 저축 목표를 설정하면 동기부여가 되고, 그 목표를 위해 필요한 금액을 더 쉽게 정할 수 있습니다.\n\n8. **고수익 저축 계좌 고려**: 일반 저축 계좌 대신 이자율이 높은 고수익 저축 계좌나 예금 상품에 투자해보는 것도 좋은 방법입니다.\n\n9. **정기적으로 저축금액 검토하기**: 자신의 재정 상황이 변화할 때마다 저축 금액을 재조정하면 더 높은 저축률을 유지할 수 있습니다.\n\n10. **추가 소득 창출**: 부업이나 프리랜서 작업 등으로 추가 소득을 창출하여 이 금액을 모두 저축하는 것도 좋은 방법입니다.\n\n이와 같은 전략들을 통해 체계적으로 저축을 늘릴 수 있습니다. 중요한 것은 지속적인 관리와 목표 설정입니다.'

- 사용자가 첫번째 질문을 입력합니다.
- 사용자가 이전에 했던 질문과 AI의 응답이 체인에 자동으로 기록되어, 다음 질문에 활용될 수 있도록 설정됩니다.

In [11]:
# 새로운 입력 메시지를 추가하고 체인 실행
chain_with_message_history.invoke(
    {"input": "방금 내가 뭐라고 했나요?"},
    {"configurable": {"session_id": "unused"}}
).content

'당신은 "저축을 늘리기 위해 무엇을 할 수 있나요?" 라고 질문하셨습니다. 그에 대해 저축을 늘릴 수 있는 여러 가지 방법을 제안했습니다. 추가적인 질문이나 필요한 정보가 있다면 언제든지 말씀해 주세요!'

## 대화 이력 요약 및 트리밍
- 챗봇이 긴 대화를 처리할 때, 이전 메시지를 모두 기억하는 것은 비효율적일 수 있습니다.
- 시간이 지날수록 대화 이력이 길어져 모델이 처리해야 할 정보량이 증가하면서 응답속도가 느려지고 불필요한 리소스가 사용될 수 있습니다.
- 오래된 메시지를 요약하거나, 삭제하여 모델이 처리할 메시지 수를 줄일 수 있습니다.

### 메시지 트리밍
- AI 모델이 대화를 처리할 때, 처리해야 할 정보의 양을 줄여 더 빠르게 효율적으로 응답할 수 있도록 하는 방법입니다.
- 가장 최근의 몇개 메시지만 남기고, 오래된 메시지는 삭제하는 방식으로 이뤄집니다.

In [12]:
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# 메시지 트리밍 유틸리티 설정
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

- 이 함수는 컨텍스트 윈도우 내에서 처리할 수 있는 토큰 수를 기반으로 메시지를 트리밍합니다.
- strategy="last"는 가장 최근 메시지를 기준으로, max_tokens=2는 메시지를 2개만 남깁니다.

In [13]:
# 트리밍된 대화 이력과 함께 체인 실행
chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer) # 대화 이력 트리밍
    | prompt
    | chat
)

- itemgetter를 통해 이전에 저장된 대화 이력을 불러오고, 이를 트리밍하여 prompt와 chat 모델에 전달합니다.

In [14]:
# 트리밍된 대화 이력을 사용하는 체인 설정
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [15]:
# 새로운 대화 내용을 추가하고 체인 실행
chain_with_trimmed_history.invoke(
    {"input": "저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?"},
    {"configurable": {"session_id": "finance_session_1"}}
)

AIMessage(content='5년 내에 집을 사기 위한 재정 계획을 세우기 위해서는 다음과 같은 몇 가지 단계를 고려해야 합니다:\n\n1. **목표 설정**:\n   - 원하는 집의 가격을 조사하세요. 주택 시장, 위치, 유형 등을 고려하여 예산을 설정합니다.\n   - 중간 목표인 5년 후 금액 외에도 주택 구매에 관련된 추가 비용(세금, 수수료, 기타 비용 등)을 계산하세요.\n\n2. **예산 수립**:\n   - 현재 소득과 지출을 분석하여 매달 저축할 수 있는 금액을 파악합니다.\n   - 지출을 줄이거나 추가 수입을 창출할 방법을 찾아보세요. 필요에 따라 불필요한 지출을 줄이거나 부업을 고려할 수 있습니다.\n\n3. **저축 계획**:\n   - 매달 저축할 금액을 정하고, 저축 계좌를 개설하여 정기적으로 납부합니다.\n   - 고이율의 저축 계좌, CD(정기예금), 또는 집 구매를 위한 특별한 저축 프로그램을 활용해 보세요.\n\n4. **신용 점수 개선**:\n   - 주택 담보 대출을 받기 위해서는 좋은 신용 점수가 필요합니다. 신용 점수를 확인하고, 개선이 필요한 경우 대출 잔액을 줄이거나 정시 납부를 통해 신용을 향상시킵니다.\n\n5. **대출 옵션 연구**:\n   - 다양한 모기지 대출 옵션을 조사하고 금리, 조건, 대출 기관을 비교합니다.\n   - 필요 시 금융 상담사나 전문가와 상담하여 최적의 대출 조건을 찾으세요.\n\n6. **시장 조사**:\n   - 부동산 시장을 지속적으로 모니터링하고, 지역 내 주택 가격 변화 및 잠재적인 접근 기회를 살펴보세요.\n\n7. **비상자금 유지**:\n   - 주택 구입 외에도 비상 자금을 유지하는 것이 중요합니다. 예기치 않은 비용을 대비하여 최소 3-6개월의 생활비를 확보하세요.\n\n이러한 단계를 고려하여 체계적으로 계획을 세우고 준비한다면 5년 내에 집을 구매하는 목표를 이룰 수 있을 것입니다. 추가적인 질문이 있으면 언제든지 말씀해 주세요!', additional_kwar

- 사용자가 질문할 때 트리밍된 대화 이력을 기반으로 AI가 적절한 응답을 생성합니다. AI는 이전 대화의 중요한 내용만 참조하고, 새로운 대화에 더 집중할 수 있습니다.

In [16]:
# 새로운 입력 메시지를 추가하고 체인 실행
chain_with_trimmed_history.invoke(
    {"input": "방금 내가 뭐라고 했나요?"},
    {"configurable": {"session_id": "finance_session_1"}}
).content

'당신은 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하는지에 대한 질문을 하셨습니다. 저는 그에 대한 재정 계획의 단계와 전략을 제안드렸습니다. 추가적인 질문이나 다른 도움이 필요하시면 말씀해 주세요!'

### 대화 요약 활용
- 대화가 길어지면 모든 대화 기록을 기억하는 것은 비효율적일 수 있습니다.
- 대화 요약을 사용하면, 이전 대화를 압축하여 중요한 정보만 남기고 AI가 새로운 질문에 응답할 때, 요약된 대화만 참조하도록 할 수 있습니다.

In [17]:
def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False

    # 대화를 요약하기 위한 프롬프트 설정
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder","{chat_history}"), # 이전 대화 이력
            ("user", "이전 대화를 요약해 주세요. 가능한 한 많은 세부 정보를 포함하십시오.") # 요약 요청 메시지
        ]
    )

    # 요약 체인 생성 및 실행
    summarization_chain = summarization_prompt | chat
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    chat_history.clear() # 요약후 이전 대화 삭제
    chat_history.add_message(summary_message) # 요약 메시지 추가

    return True

- summarize_messages() 함수는 현재 저장된 대화 기록을 요약하는 작업을 수행합니다.
- chat_history.messages 를 통해 저장된 대화 메시지를 불러옵니다.
- 요약 작업의 지침을 AI에게 전달합니다. 과거 대화 이력을 요약할 수 있게 됩니다.

In [18]:
# 대화 요약을 처리하는 체인 설정
chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

- RunnablePassthrough를 사용하여 대화 요약이 완료된 후, 이를 대화 이력과 연결하는 체인을 설정합니다.
- messages_summarized로 요약된 메시지를 대화 이력에 적용하도록 설정합니다.

In [19]:
# 요약된 대화를 기반으로 새로운 질문에 응답
print(chain_with_summarization.invoke(
    {"input": "저에게 어떤 재정적 조언을 해주셨나요?"},
    {"configurable": {"session_id": "unused"}}
))

content='아래는 저축을 늘리고 5년 내에 집을 사기 위한 재정적 계획을 세우기 위해 제안한 주요 조언 사항입니다:\n\n### 저축을 늘리기 위한 방법\n1. **예산 세우기**: 수입과 지출을 분석하고, 필요한 비용과 불필요한 지출을 파악하여 예산을 세우세요.\n2. **자동 이체 설정**: 매달 정해진 금액을 저축 계좌로 자동으로 이체하도록 설정하여 저축을 습관화하세요.\n3. **비필수 지출 줄이기**: 외식, 커피, 엔터테인먼트 등 비필수적인 지출을 줄여보세요.\n4. **할인 및 쿠폰 활용**: 쇼핑할 때 할인 쿠폰이나 프로모션을 활용해 비용을 절감하세요.\n5. **지출 트래킹 앱 사용**: 자신의 소비 패턴을 파악하기 위해 지출을 기록하고 분석하는 앱을 사용하세요.\n6. **비상금 저축**: 예기치 않은 상황에 대비해 매달 비상금을 저축하세요.\n7. **고수익 저축 계좌 고려**: 더 높은 이자율을 제공하는 저축 계좌 또는 적금 상품을 활용하세요.\n8. **정기적으로 저축금액 검토하기**: 재정 상황에 따라 저축 금액을 주기적으로 조정하세요.\n9. **추가 소득 창출**: 부업이나 프리랜스 일을 통해 추가 소득을 창출하고 이를 저축에 투자하세요.\n\n### 5년 내에 집을 사기 위한 재정 계획\n1. **목표 설정**: 원하는 주택의 대략적인 가격과 추가 비용(세금, 보험 등)을 조사하세요.\n2. **예산 수립**: 월 평균 수입을 기준으로 지출을 분석하고 월 저축 가능 금액을 파악하세요.\n3. **저축 계획**: 매달 저축 목표를 설정하고, 고이율 저축 계좌에 저축하세요.\n4. **신용 점수 개선**: 좋은 신용 점수를 유지하기 위해 빚 관리 및 신용 카드를 적절히 활용하세요.\n5. **대출 옵션 연구**: 다양한 모기지 대출 옵션과 조건을 비교하여 가장 유리한 조건을 찾으세요.\n6. **시장 조사**: 부동산 시장을 지속적으로 모니터링하고 주택 가격과 트렌드를 파악하세요.\n7. **비상자금 유지**: 비상 자금을 준비

- 요약된 대화를 기반으로 AI가 새로운 질문에 응답하도록 합니다.
- AI는 이전 긴 대화 기록을 모두 기억할 필요 없이, 핵심 정보만을 기반으로 정확한 응답을 제공할 수 있게 해줍니다.
- 요약과정에서 중요한 정보가 손실될 위험이 있으며, 요약의 정확도가 떨어질 경우 대화 맥락이 왜곡될 수 있다는 점에 주의해야 합니다.